# 🚀 Lab Extra — **Consultor de Inovação com IA** (Multiagentes + Tools + Paralelismo)

**Visão:** um *crew* que gera ideias de **novos produtos** a partir de **tendências tecnológicas** e cria um **Pitch de MVP** com plano de lançamento.  
**Agentes:** Buscador de Tendências (tool de busca), Analista de Viabilidade, Redator de Pitch.  
**Conceitos:** Tools reais (**SerperDevTool**), **paralelismo** (coleta), **síntese criativa** (MVP).  
**Saída:** Proposta de **MVP** com justificativa, riscos e **plano de lançamento 30-60-90**.

> Ritmo: **conceito → exemplo → código guiado → execução → variações**


## 🔑 Provedores — Groq (LLM) + Serper (Busca Web)
- Crie conta e chave Groq: https://console.groq.com (compatível OpenAI)  
- Crie conta e chave Serper: https://serper.dev/ (busca web)

> Defina as variáveis de ambiente abaixo (você pode colar as chaves aqui somente para o lab).


In [ ]:
import os
# === Groq (OpenAI-compatible) ===
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_GROQ"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# === Serper (web search) ===
# os.environ["SERPER_API_KEY"] = "SUA_CHAVE_SERPER"

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("OPENAI_API_BASE set?", "OPENAI_API_BASE" in os.environ)
print("SERPER_API_KEY set?", "SERPER_API_KEY" in os.environ)


## 🔧 Passo 0 — Instalar dependências
Se o Colab pedir para reiniciar, aceite e rode novamente esta célula.


In [ ]:
!pip -q install crewai crewai-tools langchain python-dotenv


## 🎯 Passo 1 — Definir Domínio, Persona e Região
Edite as variáveis para seu contexto. Exemplos de domínio: **fintech B2B**, **edtech**, **saúde digital**, **indústria 4.0**, **agritech**.


In [ ]:
DOMAIN = "fintech B2B"
PERSONA = "CFO de médias empresas no Brasil"
REGION = "Brasil"
HORIZON = "próximos 12 meses"

print("Domínio:", DOMAIN)
print("Persona:", PERSONA)
print("Região:", REGION)
print("Horizonte:", HORIZON)


Domínio: fintech B2B
Persona: CFO de médias empresas no Brasil
Região: Brasil
Horizonte: próximos 12 meses


## 🧠 Passo 2 — Conceito do fluxo
- **Coleta paralela**: tendências e sinais de concorrência (web search).  
- **Análise sequencial**: o Analista de Viabilidade sintetiza e **ranqueia 3 ideias**.  
- **Pitch final**: o Redator de Pitch gera **MVP** + **30-60-90** + riscos e métricas.


## ✅ Passo 3 — Imports, Tools e Agentes
Usaremos **SerperDevTool** para web search.


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Tool de busca
search_tool = SerperDevTool()  # usa SERPER_API_KEY

# 🔹 Agente 1 — Buscador de Tendências
trend_hunter = Agent(
    role="Buscador de Tendências Tecnológicas",
    goal=("Descobrir tendências, tecnologias emergentes, padrões de adoção e mudanças regulatórias relevantes "
          f"para {DOMAIN} no {REGION}."),
    backstory=("Você é um pesquisador objetivo que prioriza fontes confiáveis, datas recentes e impacto prático."),
    tools=[search_tool],
    allow_delegation=False,
    verbose=True
)

# 🔹 Agente 2 — Analista de Viabilidade
viability_analyst = Agent(
    role="Analista de Viabilidade",
    goal=("Transformar as tendências em 3 ideias de produto com estimativa de valor, viabilidade técnica e diferenciação. "
          "Escolher 1 ideia como **MVP** com justificativa objetiva."),
    backstory=("Você é um PM com visão de negócio e tecnologia, focado em entregas pragmáticas."),
    allow_delegation=False,
    verbose=True
)

# 🔹 Agente 3 — Redator de Pitch
pitch_writer = Agent(
    role="Redator de Pitch de Produto",
    goal=("Escrever um **Pitch de MVP** claro, persuasivo e acionável para stakeholders executivos."),
    backstory=("Você domina comunicação executiva e sabe traduzir estratégia em plano prático."),
    allow_delegation=False,
    verbose=True
)


## 🗂️ Passo 4 — Tarefas em Paralelo: Tendências + Concorrentes
Executaremos **duas buscas** em paralelo para enriquecer o contexto.


In [ ]:
trends_task = Task(
    description=(
        "Busque tendências tecnológicas e de mercado relevantes para o domínio, priorizando conteúdos "
        "dos **últimos 12–18 meses**. Traga **5–8 pontos** com: tendência/tecnologia, por que importa, e fonte (link).\n\n"
        f"Domínio: {DOMAIN}\nPersona: {PERSONA}\nRegião: {REGION}"
    ),
    expected_output="Lista em Markdown com 5–8 tendências (título, resumo curto, fonte/link, data quando possível).",
    agent=trend_hunter,
)

competitors_task = Task(
    description=(
        "Busque **concorrentes** (diretos/indiretos) e **sinais do mercado** no domínio, trazendo 4–6 itens com: "
        "empresa/solução, proposta de valor, público-alvo, e sinal relevante (lançamento, rodada, parceria, regulação).\n\n"
        f"Domínio: {DOMAIN}\nRegião: {REGION}"
    ),
    expected_output="Lista em Markdown com 4–6 concorrentes/sinais (nome, síntese da proposta, link/fonte).",
    agent=trend_hunter,
)


## 🧮 Passo 5 — Análise de Viabilidade (sequencial)
O Analista irá propor **3 ideias** e aplicar uma **matriz de pontuação** (Valor para Persona, Viabilidade Técnica, Diferenciação, Tempo de MVP).  
Critérios (0–5). Exemplo simples de **score**: `Score = Valor*0.4 + Viabilidade*0.3 + Diferenciacao*0.2 + Tempo*0.1`.


In [ ]:
viability_task = Task(
    description=(
        "Com base nas **tendências** e **concorrentes/sinais**, gere **3 ideias de produto**. Para cada uma, avalie:"
        "• **Valor para a Persona** (0–5)  • **Viabilidade Técnica** (0–5)  • **Diferenciação** (0–5)  • **Tempo para MVP** (0–5)"
        "Calcule um **Score** por ideia: `Score = Valor*0.4 + Viabilidade*0.3 + Diferenciacao*0.2 + Tempo*0.1`."
        "Escolha **1 ideia vencedora (MVP)** e explique **por que** ela é a melhor decisão **para os próximos 12 meses**."
        "Inclua uma **tabela** com as notas e scores e traga uma **curta justificativa** por critério."
    ),
    expected_output=(
        "Documento em Markdown com: 3 ideias; tabela de pontuação; MVP escolhido; justificativa resumida."
    ),
    agent=viability_analyst,
    context=[trends_task, competitors_task],
)


## 📝 Passo 6 — Redator de Pitch (síntese final)
Gera o **Pitch de MVP** com plano de lançamento **30-60-90**.


In [ ]:
pitch_task = Task(
    description=(
        "A partir do **MVP escolhido** pelo Analista, escreva um **Pitch de MVP** em Markdown com:"
        "1) **Problema & Persona** (2–3 linhas)"
        "2) **Proposta de Valor (UVP)** (1 parágrafo)"
        "3) **MVP — escopo** (3–5 funcionalidades prioritárias)"
        "4) **Arquitetura de alto nível** (breve: dados, IA, integrações)"
        "5) **Riscos & Mitigações** (3–5 bullets)"
        "6) **Plano 30-60-90** (ações por fase)"
        "7) **KPIs de sucesso** (5–7 métricas)"
        "8) **Próximos passos** (o que aprovar/financiar)"
    ),
    expected_output="Pitch de MVP em Markdown, claro, objetivo e persuasivo para stakeholders executivos.",
    agent=pitch_writer,
    context=[viability_task],
)


## 🧩 Passo 7 — Orquestração: **paralelo + sequencial**
- Paralelo: `trends_task` + `competitors_task`  
- Sequencial: `viability_task` → `pitch_task`


In [ ]:
crew = Crew(
    agents=[trend_hunter, viability_analyst, pitch_writer],
    tasks=[trends_task, competitors_task, viability_task, pitch_task],
    process=Process.hierarchical,  # Coleta em paralelo; as tarefas com context rodam na sequência
    manager_llm='gpt-4',
    verbose=True
)

print(">>> Executando Lab Extra (Inovação com IA)...")
result = crew.kickoff()

## 👀 Passo 8 — Visualizar o Pitch de MVP (Markdown)


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

## 🔁 Variações e Desafios
1) Troque `DOMAIN`, `PERSONA` e `REGION` para seu contexto real.  
2) Adicione um agente **Risk Reviewer** que critique o Pitch antes da entrega final.  
3) Acrescente uma Tool de **leitura de CSV** com feedback de clientes para influenciar a pontuação.  
4) Gere **duas versões** do Pitch: *conservadora* (baixo risco) e *arrojada* (alto impacto).  
5) Exporte o Pitch para **.md** e faça upload para um repositório público.
